### Initialize

In [2]:
import arcpy, os, sys, math
from utils.my_api import InitPath, deleteIfExist, getFieldList, reorder_fields, removeAllJoins
# 显示中文编码
'''
from imp import reload
import sys
reload(sys)
sys.setdefaultencoding('utf8')
'''

"\nfrom imp import reload\nimport sys\nreload(sys)\nsys.setdefaultencoding('utf8')\n"

In [3]:
folder_path, gdb_path = InitPath(gdb_name = "test")
print(folder_path)
print(gdb_path)

D:\COURSES\ADAS_intern\GisData
D:\COURSES\ADAS_intern\GisData\test


In [4]:
arcpy.env.workspace = gdb_path 
arcpy.env.overwriteOutput = True 

### 1.1 ??? 环境设置

In [4]:
newCoordinate = arcpy.SpatialReference(4490)
print(newCoordinate.name)
arcpy.env.outputCoordinateSystem = newCoordinate

GCS_China_Geodetic_Coordinate_System_2000


### 1.2

In [9]:
outFeatures = "grid_90.shp"
deleteIfExist(outFeatures, dele = 0)

Already exist.


In [18]:
originCoordinate = "73 0"
yAxisCoordinate = "73 10"
cornerCoordinate = "136 54"
number_rows = 2160
number_cols = 2520
labels = "NO_LABELS"
geoType = "POLYGON"

In [20]:
arcpy.CreateFishnet_management(out_feature_class = outFeatures, 
                               origin_coord = originCoordinate, y_axis_coord = yAxisCoordinate, corner_coord = cornerCoordinate, 
                               number_rows = number_rows, number_columns = number_cols, 
                               labels = labels, 
                               geometry_type = geoType)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\grid_90.shp'>

In [ ]:
getFieldList(outFeatures)

### 2

similar as 4.4

In [13]:
inFeatures = "D:\COURSES\ADAS_intern\GisData\gw.shp"
outFeatures = "shanxi_30s.shp"
deleteIfExist(outFeatures, 0)

Already exist.


In [14]:
arcpy.Select_analysis(in_features = inFeatures, 
                      out_feature_class = outFeatures, 
                      where_clause = "area_code LIKE '1401%'")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_30s.shp'>

### 3.1~3.3

In [7]:
inFeatures = "grid_90.shp" # selection applied to
targetLayer = "grid_90.lyr"
sourceLayer = "shanxi_30s.shp"

Already exist.


??? https://blog.csdn.net/gognzixiaobai666/article/details/112919955

In [8]:
arcpy.MakeFeatureLayer_management(in_features = inFeatures, out_layer = targetLayer)

<Result 'grid_90.lyr'>

In [9]:
select = arcpy.SelectLayerByLocation_management(in_layer = targetLayer, 
                                                overlap_type = "INTERSECT", # default
                                                select_features = sourceLayer, 
                                                selection_type = "NEW_SELECTION") # default

In [11]:
outFeatures = "grid_90_shanxi.shp"
arcpy.CopyFeatures_management(in_features = select, 
                              out_feature_class = outFeatures)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\grid_90_shanxi.shp'>

### 3.4

In [12]:
inFeatures = "grid_90_shanxi.shp"
clipFeatures = "shanxi_30s.shp"
outFeatures = "shanxi_90s_mlt.shp" # multi part

In [13]:
arcpy.Clip_analysis(in_features = inFeatures, 
                    clip_features = clipFeatures, 
                    out_feature_class = outFeatures)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_90s_mlt.shp'>

### 3.5

similar as 4.2

In [14]:
inFeatures = "shanxi_90s_mlt.shp"
outFeatures = "shanxi_90s.shp"

In [20]:
deleteIfExist(outFeatures)
arcpy.MultipartToSinglepart_management(in_features = inFeatures, 
                                       out_feature_class = outFeatures)

Doesn't exist.


<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_90s.shp'>

### 3.6

In [21]:
shp = "shanxi_90s.shp"
newField = "编号"
newType = "SHORT"

In [22]:
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_90s.shp'>

### 3.7

In [23]:
arcpy.CalculateField_management(in_table = shp, 
                                field = newField, 
                                expression = "!FID!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_90s.shp'>

In [24]:
with arcpy.da.SearchCursor(in_table = shp, field_names = newField) as cursor:
    # an iterable object
    vals = [row[0] for row in cursor]
    maxVal = max(vals)
    minVal = min(vals)
    print(minVal, maxVal)

(0, 1235)


In [25]:
if minVal <= 0:
    print("There are {} <= 0.".format(newField))
    with arcpy.da.UpdateCursor(in_table = shp, field_names = newField) as cursor:
        for row in cursor:
            if row[0] <= 0:
                maxVal += 1
                row[0] = maxVal
                cursor.updateRow(row)
    print("Update complete.")
else:
    print("All {} >= 1.".format(newField))

There are 编号 <= 0.
Update complete.


### 4.1

In [26]:
inFeatures = ["shanxi_90s.shp", "D:\COURSES\ADAS_intern\GisData\地质防治区划图-模板\模板_防治区划图.shp"]
outFeatures = "DZFZT_shanxi_90s_mlt.shp" # DZFZT = 地质防治图

In [27]:
arcpy.Union_analysis(inFeatures, outFeatures) # 4.1

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_shanxi_90s_mlt.shp'>

### 4.2 ???

In [28]:
inFeatures = "DZFZT_shanxi_90s_mlt.shp"
outFeatures = "DZFZT_shanxi_90s.shp"

In [29]:
arcpy.MultipartToSinglepart_management(in_features = inFeatures, 
                                       out_feature_class = outFeatures)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_shanxi_90s.shp'>

### 4.3

In [33]:
shp = "DZFZT_shanxi_90s.shp"
newField = "dz_mj"
newType = "DOUBLE"

In [34]:
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)
# getFieldList(shp)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_shanxi_90s.shp'>

算成大面积？

In [63]:
arcpy.CalculateField_management(in_table = shp, field = newField, 
                                expression = "!shape.area!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_shanxi_90s.shp'>

### 4.4

In [74]:
inFeatures = "DZFZT_shanxi_90s.shp"
outFeatures = "Export_Output_DZFZT.shp"

In [76]:
arcpy.Select_analysis(in_features = inFeatures, 
                      out_feature_class = outFeatures, 
                      where_clause = "等级值<>0")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\Export_Output_DZFZT.shp'>

### 4.5

输出在默认的default.gdb

In [77]:
inFeatures = "Export_Output_DZFZT.shp"
outFeatures = "C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZFZT_Freq"
# outFeatures = "Export_Output_DZFZT_Freq.dbf"

In [78]:
arcpy.Frequency_analysis(in_table = inFeatures, 
                         out_table = outFeatures, 
                         frequency_fields = "编号", 
                         summary_fields = "dz_mj")

<Result 'C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZFZT_Freq'>

In [79]:
print(outFeatures)
getFieldList(outFeatures)

C:\Users\lx\Documents\ArcGIS\Default.gdb\Export_Output_DZFZT_Freq


[{'name': u'OBJECTID', 'required': True, 'type': u'OID'},
 {'name': u'FREQUENCY', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'dz_mj', 'required': False, 'type': u'Double'}]

### 4.6

In [80]:
tab1 = "Export_Output_DZFZT.shp"
tab2 = outFeatures
print(tab2)

C:\Users\lx\Documents\ArcGIS\Default.gdb\Export_Output_DZFZT_Freq


In [82]:
# join 2 tables 
# on "in_data.in_field = join_table.join_field"
# only incorporate "fields" into "tab1"
arcpy.JoinField_management(in_data = tab1, 
                           in_field = "编号", 
                           join_table = tab2, 
                           join_field = "编号", 
                           fields = ["dz_mj"])

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\Export_Output_DZFZT.shp'>

In [83]:
print(tab1)
getFieldList(tab1)

Export_Output_DZFZT.shp


[{'name': u'FID', 'required': True, 'type': u'OID'},
 {'name': u'Shape', 'required': True, 'type': u'Geometry'},
 {'name': u'FID_shanxi', 'required': False, 'type': u'Integer'},
 {'name': u'Id', 'required': False, 'type': u'Integer'},
 {'name': u'ORIG_FID', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'FID_\u6a21\u677f', 'required': False, 'type': u'Integer'},
 {'name': u'\u884c\u653f\u7701', 'required': False, 'type': u'String'},
 {'name': u'\u884c\u653f\u5e02', 'required': False, 'type': u'String'},
 {'name': u'\u884c\u653f\u53bf', 'required': False, 'type': u'String'},
 {'name': u'\u66f4\u65b0\u65f6', 'required': False, 'type': u'String'},
 {'name': u'\u884c\u4e1a', 'required': False, 'type': u'String'},
 {'name': u'\u7b49\u7ea7\u503c', 'required': False, 'type': u'Integer'},
 {'name': u'dz_mj', 'required': False, 'type': u'Double'},
 {'name': u'dz_mj_1', 'required': False, 'type': u'Double'}]

### 4.7

same as 4.3

In [84]:
shp = "Export_Output_DZFZT.shp"
newField = "djz_zb"
newType = "DOUBLE"

In [88]:
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\Export_Output_DZFZT.shp'>

In [89]:
arcpy.CalculateField_management(in_table = shp, field = newField, 
                                expression = "!dz_mj! / !dz_mj_1! * !等级值!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\Export_Output_DZFZT.shp'>

### 4.8

same as 4.5

In [91]:
in_shp = "Export_Output_DZFZT.shp"
out_shp = "C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZFZT_Freq1"

In [92]:
arcpy.Frequency_analysis(in_table = in_shp, 
                         out_table = out_shp, 
                         frequency_fields = "编号", 
                         summary_fields = "djz_zb")

<Result 'C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZFZT_Freq1'>

In [93]:
arcpy.Exists(out_shp)

True

### 4.9

same as 4.6

In [94]:
tab1 = "shanxi_90s.shp"
tab2 = out_shp
print(tab2)

C:\Users\lx\Documents\ArcGIS\Default.gdb\Export_Output_DZFZT_Freq1


In [95]:
getFieldList(tab2)

[{'name': u'OBJECTID', 'required': True, 'type': u'OID'},
 {'name': u'FREQUENCY', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'djz_zb', 'required': False, 'type': u'Double'}]

In [99]:
arcpy.JoinField_management(in_data = tab1, 
                           in_field = "编号", 
                           join_table = tab2, 
                           join_field = "编号", 
                           fields = ["djz_zb"])

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_90s.shp'>

In [101]:
print(tab1)
getFieldList(tab1)

shanxi_90s.shp


[{'name': u'FID', 'required': True, 'type': u'OID'},
 {'name': u'Shape', 'required': True, 'type': u'Geometry'},
 {'name': u'Id', 'required': False, 'type': u'Integer'},
 {'name': u'ORIG_FID', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'djz_zb', 'required': False, 'type': u'Double'}]

### 4.10

same as 4.3

In [102]:
shp = "shanxi_90s.shp"
newField = "等级"
newType = "SHORT"

In [103]:
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_90s.shp'>

In [104]:
arcpy.CalculateField_management(in_table = shp, field = newField, 
                                expression = "!djz_zb!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\shanxi_90s.shp'>

### 4.11
No join active. Ignore this step

### 5.1

In [105]:
targetFeatures = "shanxi_30s.shp"
joinFeatures = "shanxi_90s.shp"
outFeatures = "DZFZT_30s_xxLevel.shp" # name "DZFZT(30s, xxLevel)" output errors?

In [106]:
deleteIfExist(outFeatures)

Doesn't exist.


In [107]:
arcpy.SpatialJoin_analysis(target_features = targetFeatures, 
                           join_features = joinFeatures, 
                           out_feature_class = outFeatures, 
                           join_operation = "JOIN_ONE_TO_ONE", 
                           join_type = "KEEP_ALL")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_30s_xxLevel.shp'>

### 6.1
添加字段

In [115]:
shp = "DZFZT_30s_xxLevel.shp"
getFieldList(shp)

[{'name': u'FID', 'required': True, 'type': u'OID'},
 {'name': u'Shape', 'required': True, 'type': u'Geometry'},
 {'name': u'Join_Count', 'required': False, 'type': u'Integer'},
 {'name': u'TARGET_FID', 'required': False, 'type': u'Integer'},
 {'name': u'oid_1', 'required': False, 'type': u'Integer'},
 {'name': u'rows', 'required': False, 'type': u'Integer'},
 {'name': u'colums', 'required': False, 'type': u'Integer'},
 {'name': u'area_code', 'required': False, 'type': u'String'},
 {'name': u'Id', 'required': False, 'type': u'Integer'},
 {'name': u'ORIG_FID', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'djz_zb', 'required': False, 'type': u'Double'},
 {'name': u'\u7b49\u7ea7', 'required': False, 'type': u'Integer'},
 {'name': u'\u7269\u7406\u4e3b', 'required': False, 'type': u'String'},
 {'name': u'\u8c03\u67e5\u5bf9', 'required': False, 'type': u'String'},
 {'name': u'\u884c\u7701', 'required': False, 'type': u'

In [111]:
arcpy.DeleteField_management(shp, "乡")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_30s_xxLevel.shp'>

In [113]:
fieldsInfo = [("物理主", "TEXT"), ("调查对", "TEXT"), ("行省", "TEXT"), ("行市", "TEXT"), 
              ("行县", "TEXT"), ("更时", "TEXT"), ("写时", "TEXT"), ("数据状", "TEXT"), 
              ("审核流", "TEXT"), ("审核状", "TEXT"), ("行业", "TEXT"), ("日期分", "TEXT"), 
              ("批次号", "TEXT"), ("乡", "TEXT"), ("等级值", "SHORT")]
# u'\u4e61' 乡

In [114]:
for name, type in fieldsInfo:
    arcpy.AddField_management(in_table = shp, 
                              field_name = name, 
                              field_type = type)

### 6.2
计算字段

In [116]:
arcpy.CalculateField_management(shp, "乡", 
                                "[area_code]")
arcpy.CalculateField_management(shp, "行省", 
                                "\"140000\"")
arcpy.CalculateField_management(shp, "行市", 
                                "Left([乡], 4) & \"00\"")
arcpy.CalculateField_management(shp, "行县", 
                                "Left([乡], 6)")
arcpy.CalculateField_management(shp, "更时", 
                                "\"20221021 11:00:00\"")
arcpy.CalculateField_management(shp, "数据状", 
                                "\"U\"")
arcpy.CalculateField_management(shp, "审核流", 
                                "\"3\"")
arcpy.CalculateField_management(shp, "审核状", 
                                "\"2\"")
arcpy.CalculateField_management(shp, "行业", 
                                "\"019\"")
arcpy.CalculateField_management(shp, "等级值", 
                                "[等级]")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_30s_xxLevel.shp'>

In [117]:
for name in ["调查对", "写时", "日期分", "批次号"]:
    arcpy.CalculateField_management(shp, name, "\"\"")

### 6.3

计算物理主键. 6位行政编码 + 第i张图 + 图内编号  
例如: 140100 + 1 + 12

In [119]:
figID = 1
expression = "!行县! + \"" + str(figID) + "\" + str(!FID!)"
print(expression)
# arcpy.GetCount_management(in_rows = shp)

!行县! + "1" + str(!FID!)


In [120]:
arcpy.CalculateField_management(shp, "物理主", 
                                expression, 
                                "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_30s_xxLevel.shp'>

### 6.4
删除多余字段

In [125]:
delFields = ["Join_Count", "TARGET_FID", "oid_1", 
             "rows", "colums", "area_code", "ORIG_FID", 
             "Shape_Length_1", "Shape_Area_1", "编号", 
             "FREQUENCY", "djz_zb", "等级", "Id"]
# delFields += ["Shape_Length", "Shape_Area"]
# getFieldList(shp)

In [128]:
# delete in batch
arcpy.DeleteField_management(in_table = shp, 
                             drop_field = ";".join(delFields)) 

<Result 'D:\\COURSES\\ADAS_intern\\GisData\\test\\DZFZT_30s_xxLevel.shp'>

### 6.5 
调整字段顺序

In [131]:
fieldOrder = [unicode(x[0], 'utf-8') for x in fieldsInfo]
# fieldOrder.insert(-1, unicode("乡", 'utf-8'))
print(fieldOrder)

[u'\u7269\u7406\u4e3b', u'\u8c03\u67e5\u5bf9', u'\u884c\u7701', u'\u884c\u5e02', u'\u884c\u53bf', u'\u66f4\u65f6', u'\u5199\u65f6', u'\u6570\u636e\u72b6', u'\u5ba1\u6838\u6d41', u'\u5ba1\u6838\u72b6', u'\u884c\u4e1a', u'\u65e5\u671f\u5206', u'\u6279\u6b21\u53f7', u'\u4e61', u'\u7b49\u7ea7\u503c']


In [135]:
[x.name for x in arcpy.ListFields(shp)]

[u'FID',
 u'Shape',
 u'\u7269\u7406\u4e3b',
 u'\u8c03\u67e5\u5bf9',
 u'\u884c\u7701',
 u'\u884c\u5e02',
 u'\u884c\u53bf',
 u'\u66f4\u65f6',
 u'\u5199\u65f6',
 u'\u6570\u636e\u72b6',
 u'\u5ba1\u6838\u6d41',
 u'\u5ba1\u6838\u72b6',
 u'\u884c\u4e1a',
 u'\u65e5\u671f\u5206',
 u'\u6279\u6b21\u53f7',
 u'\u4e61',
 u'\u7b49\u7ea7\u503c']

In [136]:
arcpy.CopyFeatures_management(shp, "tmp.shp")
reorder_fields(table = "tmp.shp", out_table = shp, 
               field_order = fieldOrder)

'DZFZT_30s_xxLevel.shp'

# ----------------------------------------------------------------------------TEST